# Chapter 6: Markov Chain Monte Carlo

## Introduction to Markov chains
Markov chain Monte Carlo (Markov Chain simulation or MCMC for short) is a Bayesian way of finding values of $\theta$ from an approximate distribution, where $\theta$ are the unknown parameters that make up the distribution. The Markov chain algorithm corrects this approximate distribution in each iteration, eventually converging to a final distribution that aligns with the posterior distribution $p(\theta|y)$. This is particularly useful when the posterior is difficult or impossible to sample making MCMC a powerful tool.

This is done by constructing a stationary Markov chain, a sequence of random variables, $\Theta_1, ... , \Theta_T$, where the $\Theta_t$ have the same distribution as the posterior. Now instead of taking independent variables, the Markov chain introduces variables $\Theta_t$ obeying the Markov property

This is done by constructing a stationary Markov chain, a sequence of random variables, $\Theta_1, ... , \Theta_T$, where the $\Theta_t$ have the same distribution as the posterior. Now instead of taking independent variables, the Markov chain introduces variables $\Theta_t$ obeying the Markov property
\begin{equation*}
	p(\Theta_{t+1} | \Theta_{t},...,\Theta_1) = p(\Theta_{t+1}|\Theta_{t}),
\end{equation*}
such that the variable $\Theta_{t+1}$ is only dependent on the most recent variable $\Theta_{t}$ and therefore, the distribution of $\Theta_{t+1}$ does not depend on how the chain got to $\Theta_{t}$, but only what this state is. Paraphrased, one could say that " What comes next is determined entirely by the present situation."
Eventually, this converging of the distribution is where the power of Markov chain simulations lies, allowing for the determination of the original distribution solely from its data. 

Markov chain simulations are frequently used when it is inefficient to find $\theta$ directly from the posterior $p(\theta|y)$. As mentioned before, we sample a new $\theta$ iteratively, with the expectation that the distribution converges more and more to $p(\theta|y)$. Generally, MCMC is exceptionally effective for high-dimensional distributions, as direct-sampling Monte Carlo methods cannot complete such analysis in a reasonable time. 

MCMC algorithms, such as Metropolis-Hasting and affine parameters, propose and accept new steps of the MCMC chain in the parameter space, allowing the chain to explore the parameter space.

## Explanation of Metropolis-Hasting
The simplest method of simulating a Markov chain is the Metropolis-Hastings algorithm, which itself relies on the more basic Metropolis algorithm. This we will inquire about first.

The Metropolis algorithm expands on a random walk with a rule that accepts or rejects the sampled data to converge the sampled data to the specified target distribution. This accepting and rejecting is based on a given rule. The algorithm works as follows:
1. Draw a starting point $\theta^0$ with $p(\theta^0|y) > 0$ from a certain starting distribution $p_0(\theta)$. For example, $p_0(\theta) \sim \mathcal{N}(\Theta_{t-1}, \sigma^2)$ if $\theta \in \mathbb{R}$ \citep{nagler2021statistics}.

2. For $t = 1, 2, \ldots$:

   a. Sample a proposal $\theta^\*$ from a proposal distribution $J_t(\theta^\*|\theta^{t-1})$ at time $t$. We require $J_t(\theta_a|\theta_b) = J_t(\theta_b|\theta_a)$ for all $\theta_a$, $\theta_b$, and $t$ for now.

   b. Calculate the ratio of the densities of $\theta_a$ and $\theta_b$:

      $$
      r = \frac{p(\theta_a|y)}{p(\theta_b|y)} = \frac{p(\theta^*|y)}{p(\theta^{t-1}|y)}.
      $$

   c. Set 

      $$
      \theta^t = \begin{cases}
         \theta^* & \text{with probability } \min(r, 1), \\
         \theta^{t-1} & \text{otherwise}.
      \end{cases}
      $$

To put this into words; a value of $\theta^*$ is sampled from a previous distribution, and if the ratio of densities is lower than a random uniform variable, the proposed state is rejected and $\theta^t$ = $\theta^{t-1}$.\\

Now we return to the Metropolis-Hastings (MH) algorithm. In contrast to the Metropolis algorithm, the MH algorithm does not require $J_t(\theta^*|\theta^{t-1})$ to be symmetric. Due to this new, asymmetric proposal distribution, we must alter the ratio of densities of $\theta_a$ and $\theta_b$, which is as follows:

$$ r = \frac{p(\theta^{\ast}|y)J_t(\theta^{t-1}|\theta^{\ast})}{p(\theta^{t-1}| y)J_t(\theta^{\ast}|\theta^{t-1})} $$

For the proposal distribution, the following characteristics are favorable:
- It is easy to sample from $J(\theta^\*|\theta)$ for any $\theta$.
- The ratio of $r$ (as given by the above equation) is easy to calculate.
- The random walk does not go too slowly, i.e., the sampled values are not too close together.
- Rejected jumps do not happen often, as they hurt the performance of the algorithm.


